# Data pipeline for uniform FEEDBACK - Herschel - Spitzer data set
- Works with data from Nextcloud repository
- Objective: get all Herschel and Spitzer data with a uniform resolution, grid and file nomenclature -set by the FEEDBACK [CII] data- for easy analysis

In [1]:
import numpy as np
import astropy.io.fits as pyfits
import astropy.wcs as wcs

from reproject import reproject_interp
from scipy.ndimage import gaussian_filter

import os
from os import listdir
from os.path import isfile, join
import time

In [2]:
#### User input ####
## region
region_name = "M16"
path_name = "../data_FEEDBACK/original/{region}/".format(region = region_name) ## directory where to get the original data
path_intermediate = "../data_FEEDBACK/intermediate/" ## directory where to store intermediate results of the processing
path_final = "../data_FEEDBACK/{region}/".format(region = region_name) ## directory where to get the final data products will be stored

## Native resolution of Spitzer and Herschel data
native_res_list = [10.7, 5.6, 17.6, 23.9, 35.2, 1.7, 1.77, 1.88, 1.98] ## first 5: Herschel, last five: Spitzer
name_info_list = ["Herschel_PACS_160",  ## first 5: Herschel, last five: Spitzer
             "Herschel_PACS_70",
             "Herschel_SPIRE_250",
             "Herschel_SPIRE_350",
             "Herschel_SPIRE_500",
             "Spitzer_IRAC_3p6",
             "Spitzer_IRAC_4p5",
             "Spitzer_IRAC_5p8",
             "Spitzer_IRAC_8",
            ]


## Target resolution of the final data
target_res = 20. ## arcsec
#smoothRes = 45. ## arcsec (preparation for GUSTO)

########################

In [3]:
## Collect the Herschel and Spitzer data
file_names = [f for f in listdir(path_name) if isfile(join(path_name, f))]

## sort the file names and print names and resolution for verification
file_names = sorted(file_names)
for i, name in enumerate(file_names):
    print("The assumed native resolution = {res}, for file {name} \n".format(res = native_res_list[i], name = name))

The assumed native resolution = 10.7, for file M16_Herschel_PACS_160_res10p7_grid2p85_large_centered.fits 

The assumed native resolution = 5.6, for file M16_Herschel_PACS_70_res5p6_grid1p4_large_centered.fits 

The assumed native resolution = 17.6, for file M16_Herschel_SPIRE_250_res17p6_grid6_large_centered.fits 

The assumed native resolution = 23.9, for file M16_Herschel_SPIRE_350_res23p9_grid10_large_centered.fits 

The assumed native resolution = 35.2, for file M16_Herschel_SPIRE_500_res35p2_grid14_large_centered.fits 

The assumed native resolution = 1.7, for file M16_Spitzer_IRAC_3p6_res1p7_grid1p2_large_centered.fits 

The assumed native resolution = 1.77, for file M16_Spitzer_IRAC_4p5_res1p77_grid1p2_large_centered.fits 

The assumed native resolution = 1.88, for file M16_Spitzer_IRAC_5p8_res1p88_grid1p2_large_centered.fits 

The assumed native resolution = 1.98, for file M16_Spitzer_IRAC_8_res1p98_grid1p2_large_centered.fits 



### End of what should be verified when running the script

In [4]:
## open the FEEDBACK [CII] fits file
hdu = pyfits.open('../data_FEEDBACK/original/CII_maps/{region}_CII_final_20_8_0p5_clean_integrated.fits'.format(region = region_name))
hdu.info()
dataCII = hdu[0].data
headerCII = hdu[0].header
wCII = wcs.WCS(headerCII)

Filename: ../data_FEEDBACK/original/CII_maps/M16_CII_final_20_8_0p5_clean_integrated.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      36   (295, 283)   float32   


#### Functions to be called in the script

In [5]:
def get_data_header(path, file_name):
    hdu = pyfits.open(path + file_name)
    hdu.info()
    return hdu[0].data, hdu[0].header

In [6]:
def smooth_data(data, sigma_conv):
    V = data.copy()
    V[np.isnan(data)] = 0
    VV = gaussian_filter(V, sigma=sigma_conv)
    
    W = 0*data.copy() + 1
    W[np.isnan(data)] = 0
    WW = gaussian_filter(W,sigma=sigma_conv)

    return VV/WW

#### End of function definitions

In [7]:
## loop over all data files. First regrid and then smooth the data to the target resolution and grid
for i, file_name in enumerate(file_names):
    ## get the data file and header information
    data, header = get_data_header(path_name, file_name)
    
    ## get the pixel size in arcseconds
    pix_size = header["CDELT2"]*3600. ## 3600 to convert from degree units to arcsecond units
    
    #### first smooth the data ####
    if(native_res_list[i] < target_res):
        start_time = time.time()
        print("Smoothing the data of {file} with an assumed native resolution {res}".format(file = file_name, res = native_res_list[i]))
        
        ## calculate the needed Gaussian convolution kernel in pixel size values
        sigma_conv = np.sqrt((target_res/2.35)**2 - (native_res_list[i]/2.35)**2) / pix_size ## /2.35 FWHM -> sigma
        
        ## smooth the data
        data_smooth = smooth_data(data, sigma_conv)
        
        ## store the smoothed data in the intermediate directory
        new_hdu = pyfits.PrimaryHDU(data_smooth, header)
        new_hdu.writeto("{path}smoothed_{file}".format(path = path_intermediate, file = file_name), overwrite = True)
        
        end_time = time.time()
        print(" -> Done in a total run time of {time} seconds \n".format(time = end_time - start_time))
    else:
        print("No smoothing done \n")
        new_hdu = pyfits.PrimaryHDU(data, header)
        new_hdu.writeto("{path}smoothed_{file}".format(path = path_intermediate, file = file_name), overwrite = True)
        
    
    #### perform the regridding on the smoothed data ####
    start_time = time.time()
    print("Starting the regridding on {file}".format(file = file_name))
    
    ## open HDU to perform regridding on
    hdu = pyfits.open("{path}smoothed_{file}".format(path = path_intermediate, file = file_name))
    
    ## perform reprojection on the [CII] grid
    array, footprint = reproject_interp(hdu, headerCII)
    
    ## prepare the header to store the regridded map
    new_header = headerCII.copy()
    new_header['BUNIT'] = 'MJy/sr'
    new_header['LINE'] = name_info_list[i]
    
    ## save the regridded map
    new_hdu = pyfits.PrimaryHDU(array, new_header)
    if(os.path.exists(path_final) == False):
        os.mkdir(path_final)
    new_hdu.writeto("{path}/{region}_{info}_res{res}_grid{grid}.fits".format(path = path_final, region = region_name, info = name_info_list[i], res = int(target_res + 0.5), grid = int(headerCII["CDELT2"]*3600. + 0.5)))
    
    end_time = time.time()
    print(" -> Done in a total run time of {time} seconds \n".format(time = end_time - start_time))
    print(" ")
        
    
    

Filename: ../data_FEEDBACK/original/M16/M16_Herschel_PACS_160_res10p7_grid2p85_large_centered.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      34   (1058, 1058)   float32   
Smoothing the data of M16_Herschel_PACS_160_res10p7_grid2p85_large_centered.fits with an assumed native resolution 10.7
 -> Done in a total run time of 0.12635588645935059 seconds 

Starting the regridding on M16_Herschel_PACS_160_res10p7_grid2p85_large_centered.fits
 -> Done in a total run time of 0.11711382865905762 seconds 

 
Filename: ../data_FEEDBACK/original/M16/M16_Herschel_PACS_70_res5p6_grid1p4_large_centered.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      34   (2156, 2156)   float32   
Smoothing the data of M16_Herschel_PACS_70_res5p6_grid1p4_large_centered.fits with an assumed native resolution 5.6
 -> Done in a total run time of 0.6045899391174316 seconds 

Starting the regridding on M16_Hersc

<ipython-input-6-1a59901ea807>:10: RuntimeWarning: invalid value encountered in true_divide
  return VV/WW


Smoothing the data of M16_Spitzer_IRAC_4p5_res1p77_grid1p2_large_centered.fits with an assumed native resolution 1.77
 -> Done in a total run time of 1.0478918552398682 seconds 

Starting the regridding on M16_Spitzer_IRAC_4p5_res1p77_grid1p2_large_centered.fits
 -> Done in a total run time of 0.15777111053466797 seconds 

 
Filename: ../data_FEEDBACK/original/M16/M16_Spitzer_IRAC_5p8_res1p88_grid1p2_large_centered.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU     111   (2520, 2516)   float64   
Smoothing the data of M16_Spitzer_IRAC_5p8_res1p88_grid1p2_large_centered.fits with an assumed native resolution 1.88
 -> Done in a total run time of 1.047229290008545 seconds 

Starting the regridding on M16_Spitzer_IRAC_5p8_res1p88_grid1p2_large_centered.fits
 -> Done in a total run time of 0.16358613967895508 seconds 

 
Filename: ../data_FEEDBACK/original/M16/M16_Spitzer_IRAC_8_res1p98_grid1p2_large_centered.fits
No.    Name      Ver    T